<a href="https://colab.research.google.com/github/GabaeOh/study_data_analytics/blob/main/codes/NLP/healthapp_review_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

refer : !pip install python-mecab-ko pyLDAvis

In [ ]:
#!pip install python-mecab-ko pyLDAvis

In [ ]:
import numpy as np

In [5]:
import pandas as pd
df_review_negative = pd.read_csv('./review_negative.csv')
df_review_positive = pd.read_csv('./review_positive.csv')

In [9]:
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 346 entries, 3 to 412
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        346 non-null    int64 
 1   app               346 non-null    object
 2   review            346 non-null    object
 3   rating            346 non-null    int64 
 4   tokenized_review  346 non-null    object
 5   replaced_review   346 non-null    object
dtypes: int64(2), object(4)
memory usage: 18.9+ KB


In [7]:
df_review_negative = df_review_negative.dropna()

## LDA 분석

#### 문장 벡터화

In [10]:
from gensim.corpora import Dictionary

In [12]:
dictionary = Dictionary(df_review_negative[['replaced_review']].values)
dictionary

In [13]:
dictionary.doc2bow(df_review_negative[['replaced_review']].values[3])

[(3, 1)]

In [15]:
# dictionary에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_negative[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [16]:
df_review_negative['문장벡터화'] = corpus_list

In [17]:
df_review_negative.columns

Index(['Unnamed: 0', 'app', 'review', 'rating', 'tokenized_review',
       'replaced_review', '문장벡터화'],
      dtype='object')

#### 토픽 잡기

In [35]:
from gensim.models import LdaModel

In [36]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=3)

In [37]:
lda_model.print_topics(num_words=4)

[(0, '0.017*"도움" + 0.015*"광고" + 0.011*"nan" + 0.010*"운동 운동"'),
 (1, '0.624*"nan" + 0.112*"운동" + 0.010*"감사" + 0.008*"추천"'),
 (2, '0.083*"최고" + 0.025*"뱃살" + 0.021*"nan" + 0.015*"조아"')]

#### 토픽 분석
-  review_positive.csv 토픽 분석
- 토픽 수는 3으로 가정

In [30]:
# 'replaced_review' 열의 모든 값을 문자열로 변환
df_review_positive['replaced_review'] = df_review_positive['replaced_review'].astype(str)

In [31]:
dictionary = Dictionary(df_review_positive[['replaced_review']].values)
dictionary

In [32]:
# dictionary에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_positive[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [33]:
df_review_positive['문장벡터화'] = corpus_list

In [34]:
df_review_positive[:3]

,Unnamed: 0,app,review,rating,tokenized_review,replaced_review,문장벡터화
0,1000,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,👍,5,NaN,nan,"[(0, 1)]"
1,1001,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,힘들지만 살이 빠지는것 같아요. 땀이 많이나요 진짜 말을 할수가 없어...,5,최고,최고,"[(1, 1)]"
2,1002,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,너무좋아요,5,NaN,nan,"[(0, 1)]"


In [38]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=3)

In [39]:
lda_model.print_topics(num_words=4)

[(0, '0.216*"운동" + 0.021*"뱃살" + 0.018*"도움" + 0.015*"추천"'),
 (1, '0.697*"nan" + 0.011*"감사" + 0.009*"광고" + 0.006*"강추"'),
 (2, '0.062*"최고" + 0.012*"운동 최고" + 0.010*"nan" + 0.008*"유용"')]